In [ ]:
# Hier nichts ändern!
import sys
sys.path.append("../code")
from setup_AB5 import *

<img src = "../figs/cammp.png" style="height:5em; width:auto" align="right"> 

# Netflix | Arbeitsblatt 5: Anwendung auf den Netflixdatensatz  <i class="fa fa-film" style="font-size:26px"></i>


Ziel ist es, die gefundene Zerlegung auch auf unbekannte Daten (Filme, die ein Nutzer noch nicht gesehen hat) anzuwenden und passende Empfehlungen auszusprechen. Um zu bewerten, wie gut sich unser Vorgehen für unbekannte Daten eignet, nutzen wir die folgende Strategie aus dem Bereich des **überwachten Maschinellen Lernens** (engl. *supervised learning*):
    
Auf diesem Arbeitsblatt wenden wir das Optimierungsverfahren auf einen Teil des originalen Netflixdatensatzes und damit auf einen größeren Datensatz an. Dabei werden wir die einzelnen Schritte aus der Animation nacheinander ausführen.



![SegmentLocal](../figs/Test_Train_Split_gif.gif "segment")

<div id="information" class="alert alert-success">
    <h2><i class="fas fa-info" style="font-size:36px"></i> &nbsp;  Hinweis zur Animation</h2>
    Bei Bedarf findest du auf diesem <a href="../help/info_ML.ipynb">Infoblatt</a> eine kurze Beschreibung der Animation.
</div>

<div id="information" class="alert alert-success">
    <h2><i class="fas fa-info" style="font-size:36px"></i> &nbsp;  Big Data</h2>
    Beim Maschinellen Lernen arbeitet man normalerweise  mit  großen Datenmengen (Stichwort <strong>Big Data</strong>). 
Das Verfahren hat dann mehr Daten, aus denen es wesentliche Eigenschaften der Nutzer und der Filme "lernen" und auf unbekannte Filme übertragen kann.


<strong><i class="fa fa-hourglass" style="font-size:20px"></i> Wir arbeiten nun mit größeren Datenmengen. Rechnungen werden deswegen deutlich länger dauern. <i class="fa fa-hourglass" style="font-size:20px"></i><strong>
</div>

## 1) Training und Testen

<div id="exercise" class="alert alert-info">
    <h3><i class="fa fa-laptop" style="font-size:30px"></i> <br><br>a) Daten laden  </h3>
    Führe den folgenden Code aus. Die Daten des Netflixdatensatzes werden geladen.
</div>

In [ ]:
# Hier nichts ändern!
movieIndex, userIndex, ratings = load_netflix_data()
print_dataset_info(movieIndex, userIndex, ratings)

<div id="exercise" class="alert alert-info">
    <h3><i class="fa fa-laptop" style="font-size:30px"></i> <br><br>b) Auswahl eines Teildatensatzes  </h3>
    Um nicht direkt auf dem kompletten, sehr großen Datensatz zu arbeiten, wählen wir einen Teil der Daten aus: 
Wir betrachten von allen Filmen in dem Datensatz nur die Filme, die mindestens von <strong>120000</strong> Nutzern bewertet wurden. Da der Datensatz immer noch sehr groß ist und  Rechnungen sehr lange dauern würden, wählen wir von den verbleibenden Nutzern zufällig <strong>n_user</strong> Nutzer aus.
Führe dazu den folgenden Code aus.
</div>

In [ ]:
n_ratings = 120000
n_user = 6500

# Laden der Filme mit mindestens n_ratings Bewertungen
movie_subset, user_subset, ratings_subset = subset_top_movies(movieIndex, userIndex, ratings,n_ratings)

# Zufällige Auswahl von n_user Nutzern
movie_subset, user_subset, ratings_subset = subset_random_users(movie_subset, user_subset, ratings_subset, n_user)
print_dataset_info(movie_subset, user_subset, ratings_subset)

<div id="exercise" class="alert alert-info">
    <h3><i class="fa fa-laptop" style="font-size:30px"></i> <br><br>c) Unterteilung in Test- und Trainingsdaten   </h3>
    Führe den folgenden Code aus. Die Daten werden in  Trainings- und Testdaten unterteilt.  Ein Auszug aus der Trainings- und der Testmatrix werden ausgeben. Üblicherweise werden mehr Daten für das Training (z.B. 80-90 % der bekannten Daten) als für das Testen genutzt. Die Auswahl der Werte, die in die Testmatrix übernommen werden, erfolgt zufällig.
</div>

In [ ]:
# Festlegung der Größe des Testdatensatzes in Prozent
percentage = 10

# Hier nicht ändern!
Rtrain, Rtest= split_data(movie_subset, user_subset, ratings_subset, percentage/100) 
print_test_train_info(Rtrain, Rtest)
print_train_test(Rtrain, Rtest, 20, 10)

<div id="exercise" class="alert alert-info">
    <h3><i class="fa fa-laptop" style="font-size:30px"></i> <br><br>d) Berechnung einer Zerlegung auf den Trainingsdaten <a id='A1_Schritt2'></a>   </h3>
    Führe den folgenden Code aus. Mit dem Optimierungsverfahren vom vorherigen Arbeitsblatt wird eine Zerlegung der Trainingsmatrix berechnet. Dir wird zudem der <strong>Fehler auf den Trainingsdaten</strong> (kurz: <strong>Trainingsfehler</strong>)  angezeigt.
</div>

In [ ]:
# Anzahl der Wiederholungen:
iterations = 20

# Anzahl der betrachteten Eigenschaften
features = 10

# Auswahl des Optimierungsvefahrens
# solver = 'SGD' ODER 'ALS'

# Regularisierungsparameter
l = 0.5
# Hier nichts ändern!

U, M, P, train_loss = als_with_mask(Rtrain, features, l, iterations)
print_train_error(train_loss)

<div id="exercise" class="alert alert-info">
    <h3><i class="fa fa-laptop" style="font-size:30px"></i> <br><br>e) Ausgabe der Vorhersagematrix  </h3>
    Führe den folgenden Code aus. Das Produkt $U \cdot M$ (d. h. unsere Vorhersagematrix) wird berechnet. Ein Auszug der Vorhersagematrix wird ausgegeben. Zum Vergleich werden erneut die Trainingsmatrix und die Testmatrix angezeigt.
</div>

In [ ]:
# Ausgabe der Vorhersagematrix
print_train_test_pred(Rtrain, Rtest, P, 20, 9)

<div id="exercise" class="alert alert-info">
    <h3><i class="fa fa-laptop" style="font-size:30px"></i> <br><br>f)  Berechnung des Fehlers auf den Testdaten  </h3>
    Führe den folgenden Code aus. Der quadratisch gemittelte <strong>Fehler auf den Trainingsdaten</strong> (kurz: <strong>Trainingsfehler</strong>)  wird berechnet.
</div>

In [ ]:
# Berechnung des Fehlers auf den Testdaten
test_loss = calcError(Rtest, P)
print_test_error(test_loss)

<div id="information" class="alert alert-success">
    <h2><i class="fas fa-info" style="font-size:36px"></i> &nbsp;  Hinweis zur Animation</h2>
    Vor der Netflix Challenge erzielte das Empfehlungssystem von Netflix einen Mittleren Quadratischen Fehler von <strong>0.9073</strong>! Diesen galt es auf einem unbekannten Datensatz um rund 10 % zu verbessern, um den Hauptgewinn von 1 Million Dollar zu gewinnen.
</div>

## 2) Parameterstudie

<div id="exercise" class="alert alert-info">
  <h3><i class="fa fa-pencil-square-o" style="font-size:36px"></i> & <i class="fa fa-laptop" style="font-size:36px"></i>&nbsp; <br><br> a) Parameter betrachten </h3>
Welchen Einfluss haben die beiden Parameter
    <ul>
        <li>Anzahl der Eigenschaften `features`</li>
        <li>Regularisierungsparameter `lambda`</li>
    </ul>
    auf den Trainings- bzw. Testfehler? Variiere dazu in den Codefeldern von <strong>1) d), e) und f)</strong> nacheinander die beiden Parameter, wie in der Tabelle angegeben. Lasse das Verfahren jeweils erneut durchlaufen und notiere deine Ergebnisse. Mit welchen Werten erhälst du den kleinsten Fehler auf den Testdaten?
</div>

In [ ]:
parameter_table()

## 3) Demo um Empfehlungssystem zu testen

<div id="exercise" class="alert alert-info">
    <h3><i class="fa fa-laptop" style="font-size:30px"></i> <br><br>a) Ausgabe der Film-Empfehlung für einen beliebigen User </h3>
    In dieser Aufgabe kannst du dir die Filmempfehlung für einen beliebigen User ausgeben lassen. 
Dazu wird in der Zeile der Vorhersage-Matrix des ausgewählten Nutzers der Film herausgesucht, bei dem das vorhergesagte Rating den höchsten Wert hatte. <br>
    Gib für <strong>None</strong> die Zeilennummer eines beliebigen Nutzers ein. Führe den Code aus dann aus.
</div>

In [ ]:
# Eingabe der Zeilennummer eines beliebigen Nutzers.
user = NaN; 

# Hier nichts ändern!
compute_recommendation(P, user)


###  Schon fertig?


#### Zusatz 1: Eigene Empfehlungen

Auf diesem [Zusatzblatt](./AB_myRecommendations.ipynb) kannst du selbst Bewertungen für verschiedene Filme abgeben und dir persönliche Empfehlungen ausgeben lassen.


<div style="text-align:left">
<img src="../figs/personal_recommendation.jpeg" width="180", class="center">
</div>

#### Zusatz 2: Video des Gewinnerteams <i class="fas fa-video"></i>

Schaue dir [hier](./Zusatzmaterial/YouTubeVideo_WinnerTeam.ipynb) ein Video an, in dem das Gewinnerteams der Netflix Challenge von seinen Erfahrungen berichtet.


<div style="text-align:left">
<img src="../figs/winner_video.png" width="200", class="center">
</div>

#### Zusatz 3: Andere Modellierungsansätze
Neben der Matrix Faktorisierung gibt es noch andere Ansätze für Empfehlungssysteme. Beispielsweise sogenannte Nachbarschaftsmethoden. Bei diesen wird  modelliert, wie ähnlich sich verschiedene Nutzer sind. Diese Ähnlichkeiten werden dann ausgenutzt, um einem User neue Filme zu empfehlen. 
* _Überlegt, wie man Ähnlichkeiten zwischen verschiedenen Usern ausgehend von der Ratingmatrix messen kann!_



# Diskussion im Plenum <i class="fa fa-comments-o" style="font-size:36px"></i> 
**Wir werden die Ergebnisse von diesem Arbeitsblatt gemeinsam diskutieren!**

____
<a rel=“license” href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br /> Dieses Werk ist lizenziert unter einer <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Namensnennung - Weitergabe unter gleichen Bedingungen 4.0 International Lizenz</a>  
Autorin: Sarah Schönbrodt